In [1]:
import numpy as np

Создадим массив всех элементов поля $GF(3^2)$ в векторном виде.

In [2]:
gf = []
for i in range(3):
    for j in range(3):
        gf.append([i, j])
gf = np.array(gf)
gf

array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2],
       [2, 0],
       [2, 1],
       [2, 2]])

In [3]:
# Сложение произвольного количества элементов в поле GF(3^2)
def sum32(*args):
    res = args[0].copy()
    for i in range(1, len(args)):
        res = np.array([(res[0] + args[i][0]) % 3, (res[1] + args[i][1] % 3)])
    return res % 3

# Произведение произвольного количества элементов в поле GF(3^2)
def mul32(*args):
    res = args[0].copy()
    for i in range(1, len(args)):
        res = np.array([((res[0] * args[i][1]) % 3 + (res[1] * args[i][0]) % 3) % 3, ((res[1] * args[i][1]) % 3 - (res[0] * args[i][0]) % 3) % 3])
    return res

In [4]:
print('Таблица сложения в GF(3^2):')
for a in gf:
    for b in gf:
        print(sum32(a, b), end='  ')
    print()

Таблица сложения в GF(3^2):
[0 0]  [0 1]  [0 2]  [1 0]  [1 1]  [1 2]  [2 0]  [2 1]  [2 2]  
[0 1]  [0 2]  [0 0]  [1 1]  [1 2]  [1 0]  [2 1]  [2 2]  [2 0]  
[0 2]  [0 0]  [0 1]  [1 2]  [1 0]  [1 1]  [2 2]  [2 0]  [2 1]  
[1 0]  [1 1]  [1 2]  [2 0]  [2 1]  [2 2]  [0 0]  [0 1]  [0 2]  
[1 1]  [1 2]  [1 0]  [2 1]  [2 2]  [2 0]  [0 1]  [0 2]  [0 0]  
[1 2]  [1 0]  [1 1]  [2 2]  [2 0]  [2 1]  [0 2]  [0 0]  [0 1]  
[2 0]  [2 1]  [2 2]  [0 0]  [0 1]  [0 2]  [1 0]  [1 1]  [1 2]  
[2 1]  [2 2]  [2 0]  [0 1]  [0 2]  [0 0]  [1 1]  [1 2]  [1 0]  
[2 2]  [2 0]  [2 1]  [0 2]  [0 0]  [0 1]  [1 2]  [1 0]  [1 1]  


In [5]:
print('Таблица умножения в GF(3^2):')
for a in gf:
    for b in gf:
        print(mul32(a, b), end='  ')
    print()

Таблица умножения в GF(3^2):
[0 0]  [0 0]  [0 0]  [0 0]  [0 0]  [0 0]  [0 0]  [0 0]  [0 0]  
[0 0]  [0 1]  [0 2]  [1 0]  [1 1]  [1 2]  [2 0]  [2 1]  [2 2]  
[0 0]  [0 2]  [0 1]  [2 0]  [2 2]  [2 1]  [1 0]  [1 2]  [1 1]  
[0 0]  [1 0]  [2 0]  [0 2]  [1 2]  [2 2]  [0 1]  [1 1]  [2 1]  
[0 0]  [1 1]  [2 2]  [1 2]  [2 0]  [0 1]  [2 1]  [0 2]  [1 0]  
[0 0]  [1 2]  [2 1]  [2 2]  [0 1]  [1 0]  [1 1]  [2 0]  [0 2]  
[0 0]  [2 0]  [1 0]  [0 1]  [2 1]  [1 1]  [0 2]  [2 2]  [1 2]  
[0 0]  [2 1]  [1 2]  [1 1]  [0 2]  [2 0]  [2 2]  [1 0]  [0 1]  
[0 0]  [2 2]  [1 1]  [2 1]  [1 0]  [0 2]  [1 2]  [0 1]  [2 0]  


In [6]:
# Поиск обратного элемента поля GF(3^2) перебором
def get_inv32(a):
    for b in gf:
        if all(mul32(a, b) == [0, 1]):
            return b

# деление 2 элементов поля GF(3^2)
def div32(a, b):
    return mul32(a, get_inv32(b))

In [7]:
# функция деления многочлена на многочлен столбиком с коэффициентами в поле GF(3^2)
def poly_div(poly, div):
    if len(div) > len(poly):
        raise ValueError
    answ = [0] * len(poly)

    p_copy = poly.copy()
    while len(p_copy) >= len(div):
        ind = len(p_copy) - len(div)
        answ[ind] = div32(p_copy[0], div[0])

        s = [mul32(answ[ind], div[i]) for i in range(len(div))]
        for i in range(ind):
            s.append([0, 0])

        s = np.array(s)
        for i in range(len(p_copy)):
            p_copy[i] = sum32(p_copy[i], -s[i])

        i = 0
        while i < len(p_copy) and all(p_copy[i] == 0):
            i += 1

        if i == len(p_copy):
            return np.zeros((2, ))
        else:
            p_copy = p_copy[i:]

    return p_copy

Будем представлять каждый полином, как вектор кэффициентов, т.е. $a_n x^n + a_{n-1} x^{n - 1} + ... + a_1 x + a_0$ представим в виде массива $[a_n, a_{n-1}, ..., a_1, a_0]$. Так как мы ищем полином 2 степени вида $x^2 + sx + k; s, k \in GF(3^2)$, неприводимый в $GF(3^2)$, то сосздадим массив polynomials всех полиномов степени 1, которые могут являться делителями полинома $x^2 + sx + k$.

In [8]:
polynomials = []
for i in range(1, len(gf)):
    polynomials.append([gf[i], [0, 0]])
    
for i in range(1, len(gf)):
    for j in range(1, len(gf)):
        polynomials.append([gf[i], gf[j]])

polynomials = np.array(polynomials)

Переберем все возможные $s, k \in GF(3^2)$ и найдем подходящие полиномы вида $x^2 + sx + k$.

In [13]:
for i in range(1, len(gf)):
    for j in range(1, len(gf)):
        poly = np.array([[0, 1], gf[i], gf[j]])
        for div in polynomials:
            if all(poly_div(poly, div).ravel() == 0):
                break
        else:
            print(f'Полином неприводимый при s = {gf[i]}   k={gf[j]}')

Полином неприводимый при s = [0 1]   k=[1 0]
Полином неприводимый при s = [0 1]   k=[1 2]
Полином неприводимый при s = [0 1]   k=[2 0]
Полином неприводимый при s = [0 1]   k=[2 2]
Полином неприводимый при s = [0 2]   k=[1 0]
Полином неприводимый при s = [0 2]   k=[1 2]
Полином неприводимый при s = [0 2]   k=[2 0]
Полином неприводимый при s = [0 2]   k=[2 2]
Полином неприводимый при s = [1 0]   k=[1 0]
Полином неприводимый при s = [1 0]   k=[1 1]
Полином неприводимый при s = [1 0]   k=[2 0]
Полином неприводимый при s = [1 0]   k=[2 1]
Полином неприводимый при s = [1 1]   k=[0 1]
Полином неприводимый при s = [1 1]   k=[0 2]
Полином неприводимый при s = [1 1]   k=[1 1]
Полином неприводимый при s = [1 1]   k=[1 2]
Полином неприводимый при s = [1 2]   k=[0 1]
Полином неприводимый при s = [1 2]   k=[0 2]
Полином неприводимый при s = [1 2]   k=[2 1]
Полином неприводимый при s = [1 2]   k=[2 2]
Полином неприводимый при s = [2 0]   k=[1 0]
Полином неприводимый при s = [2 0]   k=[1 1]
Полином не

Построим поле $GF((3^2)^2)$ по модулю полинома $x^2 + x + [1, 0]$.

In [14]:
# Сумма произвольного количества элементов в GF((3^2)^2)
def sum322(*args):
    res = args[0].copy()
    for i in range(1, len(args)):
        res = (res + args[i]) % 3
    return res % 3

# Произведение произвольного количества элементов в GF((3^2)^2)
def mul322(*args):
    res = args[0].copy()
    for i in range(1, len(args)):
        q = sum32(mul32(np.array([0, 2]), res[0], args[i][0]), mul32(res[0], args[i][1]), mul32(res[1], args[i][0]))
        p = sum32(mul32(res[1], args[i][1]), mul32(np.array([0, 2]), np.array([1, 0]), res[0], args[i][0]))
        res = np.array([q % 3, p % 3])
    return res

Пусть $g$ - корень полинома $x^2 + x + [1, 0]$. Проверим, является ли g примитивным элементом поля $GF((3^2)^2)$.

In [56]:
g = np.array([[0, 1], [0, 0]])
g_i = g.copy()
for i in range(2, 81):
    g_i = mul322(g_i, g)
    if np.all(g_i.ravel() == np.array([0, 0, 0, 1])):
        print('attention', end=' ')
    print(f'gamma^{i} = {g_i.ravel()}')

gamma^2 = [0 2 2 0]
gamma^3 = [2 1 1 0]
gamma^4 = [2 2 2 2]
gamma^5 = [0 0 1 2]
gamma^6 = [1 2 0 0]
gamma^7 = [2 1 1 1]
gamma^8 = [2 0 2 2]
gamma^9 = [0 2 0 2]
gamma^10 = [0 0 1 0]
gamma^11 = [1 0 0 0]
gamma^12 = [2 0 0 1]
gamma^13 = [1 1 0 2]
gamma^14 = [2 1 2 1]
gamma^15 = [0 0 2 2]
gamma^16 = [2 2 0 0]
gamma^17 = [1 1 1 2]
gamma^18 = [0 1 2 1]
gamma^19 = [2 0 2 0]
gamma^20 = [0 0 0 2]
gamma^21 = [0 2 0 0]
gamma^22 = [0 1 1 0]
gamma^23 = [1 2 2 0]
gamma^24 = [1 1 1 1]
gamma^25 = [0 0 2 1]
gamma^26 = [2 1 0 0]
gamma^27 = [1 2 2 2]
gamma^28 = [1 0 1 1]
gamma^29 = [0 1 0 1]
gamma^30 = [0 0 2 0]
gamma^31 = [2 0 0 0]
gamma^32 = [1 0 0 2]
gamma^33 = [2 2 0 1]
gamma^34 = [1 2 1 2]
gamma^35 = [0 0 1 1]
gamma^36 = [1 1 0 0]
gamma^37 = [2 2 2 1]
gamma^38 = [0 2 1 2]
gamma^39 = [1 0 1 0]
attention gamma^40 = [0 0 0 1]
gamma^41 = [0 1 0 0]
gamma^42 = [0 2 2 0]
gamma^43 = [2 1 1 0]
gamma^44 = [2 2 2 2]
gamma^45 = [0 0 1 2]
gamma^46 = [1 2 0 0]
gamma^47 = [2 1 1 1]
gamma^48 = [2 0 2 2]
gamma^49 = 

g не является примитивным элементом поля $GF((3^2)^2)$.

In [21]:
gf_322 = []
for i in gf:
    for j in gf:
        gf_322.append([i, j])

gf_322 = np.array(gf_322)
gf_322

array([[[0, 0],
        [0, 0]],

       [[0, 0],
        [0, 1]],

       [[0, 0],
        [0, 2]],

       [[0, 0],
        [1, 0]],

       [[0, 0],
        [1, 1]],

       [[0, 0],
        [1, 2]],

       [[0, 0],
        [2, 0]],

       [[0, 0],
        [2, 1]],

       [[0, 0],
        [2, 2]],

       [[0, 1],
        [0, 0]],

       [[0, 1],
        [0, 1]],

       [[0, 1],
        [0, 2]],

       [[0, 1],
        [1, 0]],

       [[0, 1],
        [1, 1]],

       [[0, 1],
        [1, 2]],

       [[0, 1],
        [2, 0]],

       [[0, 1],
        [2, 1]],

       [[0, 1],
        [2, 2]],

       [[0, 2],
        [0, 0]],

       [[0, 2],
        [0, 1]],

       [[0, 2],
        [0, 2]],

       [[0, 2],
        [1, 0]],

       [[0, 2],
        [1, 1]],

       [[0, 2],
        [1, 2]],

       [[0, 2],
        [2, 0]],

       [[0, 2],
        [2, 1]],

       [[0, 2],
        [2, 2]],

       [[1, 0],
        [0, 0]],

       [[1, 0],
        [0, 1]],

       [[1, 0]

In [22]:
len(gf_322)

81

gf_322 - массив, содержащий все элементы поля $GF((3^2)^2)$ в векторном представлении.

Будем искать генерирующий элемент поля $GF((3^2)^2)$ в виде $g + x, x \in GF((3^2)^2)$.

In [24]:
for el in gf_322:
    g_x = sum322(np.array([[0, 1], [0, 0]]), el)
    g_i = g_x.copy()
    for i in range(2, 81):
        g_i = mul322(g_i, g_x)
        if np.all(g_i.ravel() == np.array([0, 0, 0, 1])):
            if (i == 80):
                print(f'g + {el.tolist()} является генерирующим элементом поля', end='\n\n')
            else:
                break

g + [[0, 0], [1, 1]] является генерирующим элементом поля

g + [[0, 0], [1, 2]] является генерирующим элементом поля

g + [[0, 0], [2, 0]] является генерирующим элементом поля

g + [[0, 0], [2, 2]] является генерирующим элементом поля

g + [[0, 1], [1, 0]] является генерирующим элементом поля

g + [[0, 1], [1, 1]] является генерирующим элементом поля

g + [[0, 1], [2, 1]] является генерирующим элементом поля

g + [[0, 1], [2, 2]] является генерирующим элементом поля

g + [[1, 0], [0, 1]] является генерирующим элементом поля

g + [[1, 0], [1, 0]] является генерирующим элементом поля

g + [[1, 0], [2, 0]] является генерирующим элементом поля

g + [[1, 0], [2, 1]] является генерирующим элементом поля

g + [[1, 1], [0, 1]] является генерирующим элементом поля

g + [[1, 1], [0, 2]] является генерирующим элементом поля

g + [[1, 1], [1, 0]] является генерирующим элементом поля

g + [[1, 1], [1, 1]] является генерирующим элементом поля

g + [[1, 2], [0, 1]] является генерирующим элементом пол

Найдем все степени элемента $g + [1, 1]$.

In [25]:
g = sum322(np.array([[0, 1], [0, 0]]), np.array([[0, 0],[1, 1]]))

g_i = g.copy()
for i in range(2, 81):
    g_i = mul322(g_i, g)
    if np.all(g_i.ravel() == np.array([0, 0, 0, 1])):
        print('attention', end=' ')
    print(f'(g + [1, 1])^{i} = {g_i.ravel()}')

(g + [1, 1])^2 = [2 1 1 0]
(g + [1, 1])^3 = [2 1 0 1]
(g + [1, 1])^4 = [1 2 0 0]
(g + [1, 1])^5 = [2 2 1 1]
(g + [1, 1])^6 = [0 2 0 2]
(g + [1, 1])^7 = [2 2 0 2]
(g + [1, 1])^8 = [2 0 0 1]
(g + [1, 1])^9 = [0 2 1 0]
(g + [1, 1])^10 = [0 0 2 2]
(g + [1, 1])^11 = [2 2 1 0]
(g + [1, 1])^12 = [0 1 2 1]
(g + [1, 1])^13 = [0 1 2 2]
(g + [1, 1])^14 = [0 2 0 0]
(g + [1, 1])^15 = [2 0 1 0]
(g + [1, 1])^16 = [1 1 1 1]
(g + [1, 1])^17 = [2 0 1 1]
(g + [1, 1])^18 = [1 2 2 2]
(g + [1, 1])^19 = [1 1 2 1]
(g + [1, 1])^20 = [0 0 2 0]
(g + [1, 1])^21 = [2 0 2 1]
(g + [1, 1])^22 = [2 2 0 1]
(g + [1, 1])^23 = [2 2 2 0]
(g + [1, 1])^24 = [1 1 0 0]
(g + [1, 1])^25 = [1 2 2 1]
(g + [1, 1])^26 = [1 0 1 0]
(g + [1, 1])^27 = [1 2 1 0]
(g + [1, 1])^28 = [0 2 2 0]
(g + [1, 1])^29 = [1 0 0 1]
(g + [1, 1])^30 = [0 0 1 2]
(g + [1, 1])^31 = [1 2 0 1]
(g + [1, 1])^32 = [2 0 2 2]
(g + [1, 1])^33 = [2 0 1 2]
(g + [1, 1])^34 = [1 0 0 0]
(g + [1, 1])^35 = [0 2 0 1]
(g + [1, 1])^36 = [2 1 2 1]
(g + [1, 1])^37 = [0 2 2 1]


Составим матрицу перехода от поля $GF(3^4)$ к полю $GF((3^2)^2)$.

In [26]:
T = np.array([
    [2, 2, 0, 0],
    [1, 1, 1, 0],
    [0, 1, 1, 0],
    [1, 0, 1, 1]
])

$(g + [1, 1])^4 = [1, 2, 0, 0]$, следовательно нам необходимо найти такой неприводимый в $GF(3)$ полином 4 степени, по модулю которого мы будем строить поле  $GF(3^4)$, который будет удовлетворять равенству $T * \beta^4 = [1, 2, 0, 0] = (g + [1, 1])^4$, где $\beta$ - генерирующий элемент поля $GF(3^4)$.

Возьмем все примитивные над полем $GF(3)$ полиномы 4 степени. Пусть $x^4 + a x^3 + b x^2 +c x + d$ - примитивный в поле $GF(3^4)$ многочлен, $\beta$ - корень этого многочлена. Тогда $\beta^4 = -a \beta^3 - b \beta^2 - c \beta - d$. Массив prim_poly состоит из массивов элеметов $[a, b, c, d]$, соответствующих примитивным многочленам. Массив betas - все возможные значения $\beta^4$.

In [34]:
prim_poly = np.array([
    [0, 0, 1, 2],
    [0, 0, 2, 2],
    [1, 0, 0, 2],
    [1, 1, 2, 2],
    [1, 2, 2, 2],
    [2, 0, 0, 2],
    [2, 1, 1, 2],
    [2, 2, 1, 2],
])

betas = -prim_poly % 3
betas

array([[0, 0, 2, 1],
       [0, 0, 1, 1],
       [2, 0, 0, 1],
       [2, 2, 1, 1],
       [2, 1, 1, 1],
       [1, 0, 0, 1],
       [1, 2, 2, 1],
       [1, 1, 2, 1]])

Найдем элемент $\beta^4$, удовлетворяющий условию $T * \beta^4 = [1, 2, 0, 0]$.

In [35]:
for i in range(len(betas)):
    if np.all((T @ betas[i]) % 3 == np.array([1, 2, 0, 0])):
        print(prim_poly[i])

[1 0 0 2]


Таким образом примитивный элемент поля $GF(3^4)$, построенное по модулю примитивного многочлена x^4 + x + 2, будет удовлетворять условиям изоморфизма.

In [37]:
# функция вычисления алгебраического дополнения
def get_alg_add(A, i, j):
    return np.delete(np.delete(A, i, axis=0), j, axis=1)

# поиск определителя матрицы в поле GF(3)
def get_det(A):
    if (A.shape[0] == 1):
        return A[0] % 3
    elif (A.shape[0] == 2):
        return (A[0, 0] * A[1, 1] - A[0, 1] * A[1, 0]) % 3
    
    res = 0
    for j in range(A.shape[0]):
        res += (-1)**j * A[1, j] * get_det(get_alg_add(A, 1, j)) % 3
    return res

# поиск обратного значения в поле GF(3)
def get_inv_val(x):
    for i in range(1, 3):
        if (x * i) % 3 == 1:
            return i

In [38]:
# функция поиска обратной матрицы в поле GF(3)
def get_inv(A):
    adj_A = np.zeros(A.shape).astype(int)
    
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            adj_A[i, j] = (get_det( get_alg_add(A, i, j) ) * (-1)**(i + j))
            
    return (get_inv_val(get_det(A)) * adj_A.T) * 2 % 3

In [39]:
T_1 = get_inv(T)
T_1

array([[0, 1, 2, 0],
       [2, 2, 1, 0],
       [1, 1, 0, 0],
       [2, 1, 1, 1]])

In [40]:
# функция отображения из одного поля в другое
def Map(T, a):
    return (T @ a) % 3

In [41]:
Map(T_1, np.array([2, 2, 0, 2]))

array([2, 2, 1, 2])

In [77]:
Map(T, np.array([2, 2, 1, 2]))

array([2, 2, 0, 2])

Можно проверить условия изоморфизма для любых элементов полей.